In [26]:
from dotenv import load_dotenv
import os
from pathlib import Path
import alpaca_trade_api as ata

In [32]:
local_dir = os.path.dirname(os.path.abspath('__file__'))
parent_dir = os.path.dirname(local_dir)
config_file_path = os.path.join(parent_dir,'API.env')
load_dotenv(Path(config_file_path))

True

In [33]:
api_key = os.getenv("API_KEY")
api_secret = os.getenv("API_SECRET")
base_url = os.getenv("BASE_URL")